<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex10/patrick_ferreira/ex10_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: Patrick de Carvalho Tavares Rezende Ferreira

## Instruções:

O objetivo desse colab é entender o comportamento das seguintes variáveis importantes de treinamento:
   - Batch size
   - Learning rate
   - FLOPs (computação gasta no treinamento)
   - Tamanho do modelo

Para tanto, iremos treinar e medir a loss e acurácia de 3 modelos BERT para análise de sentimento (classificação binária) usando o dataset do IMDB (20k/5k amostras de treino/validação).

Iremos fazer os 3 x 5 x 5 = 75 treinamentos, cada um usando um modelo (dentre 3), uma learning rate (dentre 5 valores) e batch size (dentro 5 valores) diferentes.

Os modelos sugeridos a serem usados são:

*   google/bert_uncased_L-2_H-128_A-2 (BERT-tiny, 4M params, ~0.5M non-embeddings)
*   google/bert_uncased_L-4_H-256_A-4 (BERT-mini, 11M params, ~3.5M non-embeddings)
*   google/bert_uncased_L-8_H-512_A-8 (BERT-medium, 41M params, ~25M non-embeddings)

Durante cada treinamento, iremos armezenar as seguntes informações: 

    - GPU usada
    - FP16 ou 32?
    - step atual
    - tempo de treinamento até então (wall time)
    - loss de treino
    - loss de validação
    - acurácia de validação
    
Iremos gravar essas informações _várias vezes por época_. Caso os treinamentos usem GPUs diferentes, podemos ajustar o wall time com base no FLOPs das GPUs.

Ao final, iremos plotar os seguintes gráficos:

1.   batch_size vs learning rate vs melhor loss de validação para cada modelo (usar gráfico 3D ou heatmap);
2.   tempo de treinamento (wall time) vs loss de validação. Plotar uma série (curva) para cada modelo, todas no mesmo gráfico. Para gerar cada curva, usar os melhores batch size e learning rate encontrados no gráfico 1. 

Com isso conseguiremos responder às seguintes perguntas:

    - Se você tiver T horas de GPU para usar, é melhor usar o modelo tiny, mini ou medium? Verifique se existe alguma faixa de valores de T em que é melhor usar o tiny. 
    - Qual modelo demora mais para atingir a sua melhor acurácia de validação em termos de épocas. E em termos de tempo de treino, wall time?
    - Para cada X vezes que aumentamos o batch size, como que devemos ajustar a learning rate?
    - Os melhores hiperparametros são parecidos para os 3 modelos?

Notas:
- Para entender melhor como batch size e learning rate se relacionam, procure fazer a varredura com passos de 5x ou 10x. Por exemplo:
    
    learning rate = {1e-2, 1e-3, ..., 1e-6}
    
    batch size = {1, 10, 100, 1000, 10000}

- Caso o batch não caiba em memória, usar acumulo de gradiente.
- Tempos estimados de treinamento para uma época do IMDB usando uma T4:
    - BERT-tiny: menos de 1 minuto
    - BERT-mini: 3 minutos
    - BERT-medium: 10 minutos. Portanto, se treinarmos por 2 épocas, o tempo total para rodar os experimentos será de `2 épocas x 10 min x 25 treinamentos ~ 9 horas`.
- Sugerimos fazer primeiro todos os experimentos com BERT-tiny e BERT-mini. Quando souber da faixa de hiperparametros "bons", não precisa fazer os 25 treinamentos para o BERT-medium.
    - TFLOPs (FP32) de cada GPU:
        T4: 8,141
        K80: 4,113
        A100: 19,49
    - Usar time.perf_counter() para medir o wall time.

# Fixando a seed

In [1]:
import random
import re
import time

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from typing import List, Type
from tqdm.notebook import tqdm

In [2]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Nov  2 22:03:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P8    13W / 170W |    160MiB / 12288MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Preparando Dados

Primeiro, fazemos download do dataset:

In [6]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (20k exemplos) e validação (5k exemplos) artificialmente.

In [7]:
import os

max_valid = 5000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [1] * len(x_train_pos) + [0] * len(x_train_neg)
y_test = [1] * len(x_test_pos) + [0] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])


20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
0 Barry, a medical transcriptionist has his mind corroding from his job coupled with memories of an ab
0 The movie seemed to appeal me because of the new type of Pokemon Celebi. But the plot was out of cou
1 I can't remember many films where a bumbling idiot of a hero was so funny throughout. Leslie Cheung 
3 últimas amostras treino:
0 I sat down to watch a documentary about Puerto Rico, and I ended up watching one about Nuyoricans. W
1 It's a great American martial arts movie. The fighting scenes were pretty impressive for American mo
1 The appeal of ancient films like this one is that you get to see an actual moving image of life over
3 primeiras amostras validação:
1 Why is this movie not in the 250 best? This movie looks still astoundingly fresh 56 years after its 
1 I've always believed that David and Bathsheba was a film originally intended for Tyrone Power at 20t
1 I

In [8]:
### Criando classe do dataset

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, texts: List[str], labels):
        super().__init__()

        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor([0., 1.]) if self.labels[idx] else torch.tensor([1., 0.])

Dados de treino, validação e teste

In [9]:
training_dataset = MyDataset(x_train, y_train)
valid_dataset = MyDataset(x_valid, y_valid)
test_dataset = MyDataset(x_test, y_test)

print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 20000
valid examples: 5000
test examples: 25000


Testando se o modelo processa os dados corretamente

In [10]:
model = tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.train().to(device)

sample_train, _ = next(iter(DataLoader(training_dataset, batch_size=4)))

sample_train = tokenizer.batch_encode_plus(sample_train, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device)

print("model output shape: ", model(**sample_train).logits.shape)

del sample_train


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

model output shape:  torch.Size([4, 2])


TREINAMENTO

In [11]:
from torch.cuda.amp import GradScaler, autocast

use_amp = True

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        logits = model(**input_ids).logits
        logits = logits.reshape(-1, logits.shape[-1])
    loss = nn.functional.cross_entropy(logits, target_ids, )
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    with autocast(enabled=use_amp):
        logits = model(**input_ids).logits
        loss = nn.functional.cross_entropy(logits, target_ids,)
        preds = logits.argmax(dim=1)
        accuracy = (preds == target_ids.argmax(dim=1)).sum().float() / logits.shape[0]
    return loss.item(), accuracy.item()

epochs = 0.01
max_examples = int(len(training_dataset) * epochs)

rtx3060_tflops = 12.74 # https://www.techpowerup.com/gpu-specs/geforce-rtx-3060.c3682

model_list = ["google/bert_uncased_L-2_H-128_A-2",
              "google/bert_uncased_L-4_H-256_A-4",]
              # "google/bert_uncased_L-8_H-512_A-8",]
batch_sizes = [16, 32]
learning_rates = [1e-2, 1e-6]

log_df = pd.DataFrame.from_dict({
    "model_name": [],
    "batch_size": [],
    "lr": [],
    "val_loss": [],
    "best_acc": [],
    "wall_time": []
})

for model_name in model_list:
    for batch_size in batch_sizes:
        for lr in learning_rates:
            model_identifier = model_name + "_batch_size_" + str(batch_size) + "_lr_" + str(lr)
            print("Current Setup: ", model_identifier)

            model = BertForSequenceClassification.from_pretrained(model_name)
            model.train().to(device)

            eval_every_steps = 1000 * batch_sizes[0] // (batch_size // 10)

            train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
            validation_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=1, )

            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            scaler=GradScaler()

            best_validation_loss = 9999
            best_validation_acc = 0
            convergence_time = 0
            train_losses = []
            n_examples = 0
            step = 0
            pbar = tqdm(total=max_examples)
            start_time = time.time()
            while n_examples < max_examples:
                val_losses = []
                wall_times = []
                for train_input_ids, train_target_ids in train_loader:
                    train_input_ids = tokenizer.batch_encode_plus(train_input_ids, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device)
                    loss = train_step(train_input_ids, train_target_ids.to(device))
                    train_losses.append(loss)

                    if step % eval_every_steps == 0:
                        train_loss = np.average(train_losses)

                        with torch.no_grad():
                            valid_result = list(zip(*[
                                validation_step(tokenizer.batch_encode_plus(val_input_ids, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device), val_target_ids.to(device))
                                for val_input_ids, val_target_ids in validation_loader]))
                            valid_loss = np.average(valid_result[0])
                            valid_acc = np.average(valid_result[1])
                            # Checkpoint to best models found.
                            if best_validation_loss > valid_loss:
                                # Update the new best perplexity.
                                best_validation_loss = valid_loss
                                best_validation_acc = valid_acc
                                convergence_time = time.time() - start_time

                        val_losses.append(valid_loss)
                        wall_times.append((time.time()-start_time) * rtx3060_tflops)
                        print(f'{step} steps; {n_examples} examples so far; train loss: {train_loss:.2f}, valid loss: {valid_loss:.2f}')
                        train_losses = []

                    n_examples += train_input_ids.input_ids.shape[0] # Increment of batch size
                    step += 1
                    pbar.update(train_input_ids.input_ids.shape[0])
                    if n_examples >= max_examples:
                        break
            print("SALVOOOOOUUUUU")
            log_df = log_df.append({
                "model_name": model_name,
                "batch_size": batch_size,
                "lr": lr,
                "val_loss": best_validation_loss,
                "best_acc": best_validation_acc,
                "wall_time": convergence_time * rtx3060_tflops,
                "losses_list": val_losses,
                "times": wall_times,
            }, ignore_index=True)

            pbar.close()

# Restore best model (checkpoint) found
# model = torch.load("best_model.pth")

Current Setup:  google/bert_uncased_L-2_H-128_A-2_batch_size_16_lr_0.01


Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

  0%|          | 0/200 [00:00<?, ?it/s]

0 steps; 0 examples so far; train loss: 0.70, valid loss: 0.69
SALVOOOOOUUUUU
Current Setup:  google/bert_uncased_L-2_H-128_A-2_batch_size_16_lr_1e-06


/tmp/ipykernel_33841/2706262820.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append({
Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkp

...

0 steps; 0 examples so far; train loss: 0.69, valid loss: 1.19
SALVOOOOOUUUUU
Current Setup:  google/bert_uncased_L-4_H-256_A-4_batch_size_32_lr_1e-06


/tmp/ipykernel_33841/2706262820.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append({
Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkp

  0%|          | 0/200 [00:00<?, ?it/s]

0 steps; 0 examples so far; train loss: 0.69, valid loss: 0.69
SALVOOOOOUUUUU


/tmp/ipykernel_33841/2706262820.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append({


In [12]:
log_df.loc[log_df['model_name'] == "google/bert_uncased_L-2_H-128_A-2"]

,model_name,batch_size,lr,val_loss,best_acc,wall_time,losses_list,times
0,google/bert_uncased_L-2_H-128_A-2,16.0,0.010000,0.693602,0.507788,226.556708,[0.693602053121256],[226.55689306735994]
1,google/bert_uncased_L-2_H-128_A-2,16.0,0.000001,0.696576,0.469050,215.680036,[0.6965758107340755],[215.68022449970246]
2,google/bert_uncased_L-2_H-128_A-2,32.0,0.010000,0.800803,0.507166,204.462019,[0.8008026317426353],[204.4622039604187]
3,google/bert_uncased_L-2_H-128_A-2,32.0,0.000001,0.709038,0.492635,203.797734,[0.7090380723309365],[203.7979252052307]
